# Prior Distribution Generation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datetime as dt

from db_tools import read_csv_covariance, read_csv_profiles
from optimal_estimation import rgrid

%matplotlib inline

## COSMO7

- Interpolate to radiosonde launch time
- Remove bias wrt to radiosonde
- Save bias-corrected values and covariance matrix of errors

In [ ]:
def interpolate(target, dfs, hourbounds=(0, 6)):
    lower = min(min(df.index) for df in dfs)
    upper = max(max(df.index) for df in dfs)
    idx = pd.Index([
            d for d in target.index
                    if lower <= d <= upper
                    and hourbounds[0] <= d.hour <= hourbounds[1]
            ], name="valid")
    out = pd.DataFrame(index=idx.union_many(df.index for df in dfs), columns=target.columns, dtype=float)
    for df in dfs:
        out.ix[df.index] = df
    return idx, out.interpolate(method="time", axis=0).ix[idx]

In [ ]:
def cosmo_prior(raso, dfl, dfh):
    idx, itp = interpolate(raso, [dfl, dfh])
    diff = raso.ix[idx] - itp
    return itp + diff.mean(), diff.cov()

### Temperature Prior

In [ ]:
Traso = read_csv_profiles("../data/unified/T_raso.csv")
Tcosmo00 = read_csv_profiles("../data/unified/T_cosmo7+00.csv")
Tcosmo06 = read_csv_profiles("../data/unified/T_cosmo7+06.csv")
Tcosmo24 = read_csv_profiles("../data/unified/T_cosmo7+24.csv")
Tcosmo30 = read_csv_profiles("../data/unified/T_cosmo7+30.csv")

means0006, cov0006 = cosmo_prior(Traso, Tcosmo00, Tcosmo06)
means2430, cov2430 = cosmo_prior(Traso, Tcosmo24, Tcosmo30)

means0006.to_csv("../data/unified/priors/T_cosmo7+00+06_mean.csv")
means2430.to_csv("../data/unified/priors/T_cosmo7+24+30_mean.csv")
cov0006.to_csv("../data/unified/priors/T_cosmo7+00+06_cov.csv")
cov2430.to_csv("../data/unified/priors/T_cosmo7+24+30_cov.csv")

### Humidity Prior

In [ ]:
qraso = read_csv_profiles("../data/unified/qvap_raso.csv")
qcosmo00 = read_csv_profiles("../data/unified/qvap_cosmo7+00.csv")
qcosmo06 = read_csv_profiles("../data/unified/qvap_cosmo7+06.csv")
qcosmo24 = read_csv_profiles("../data/unified/qvap_cosmo7+24.csv")
qcosmo30 = read_csv_profiles("../data/unified/qvap_cosmo7+30.csv")

qraso += read_csv_profiles("../data/unified/qliq_raso.csv")
qcosmo00 += read_csv_profiles("../data/unified/qliq_cosmo7+00.csv")
qcosmo06 += read_csv_profiles("../data/unified/qliq_cosmo7+06.csv")
qcosmo24 += read_csv_profiles("../data/unified/qliq_cosmo7+24.csv")
qcosmo30 += read_csv_profiles("../data/unified/qliq_cosmo7+30.csv")

means0006, cov0006 = cosmo_prior(np.log(qraso), np.log(qcosmo00), np.log(qcosmo06))
means2430, cov2430 = cosmo_prior(np.log(qraso), np.log(qcosmo24), np.log(qcosmo30))

means0006.to_csv("../data/unified/priors/lnq_cosmo7+00+06_mean.csv")
means2430.to_csv("../data/unified/priors/lnq_cosmo7+24+30_mean.csv")
cov0006.to_csv("../data/unified/priors/lnq_cosmo7+00+06_cov.csv")
cov2430.to_csv("../data/unified/priors/lnq_cosmo7+24+30_cov.csv")

### State Vector Prior

In [ ]:
xraso = pd.concat([Traso.add_prefix("T_"), np.log(qraso).add_prefix("lnq_")], axis=1)
xcosmo00 = pd.concat([Tcosmo00.add_prefix("T_"), np.log(qcosmo00).add_prefix("lnq_")], axis=1)
xcosmo06 = pd.concat([Tcosmo06.add_prefix("T_"), np.log(qcosmo06).add_prefix("lnq_")], axis=1)
xcosmo24 = pd.concat([Tcosmo24.add_prefix("T_"), np.log(qcosmo24).add_prefix("lnq_")], axis=1)
xcosmo30 = pd.concat([Tcosmo30.add_prefix("T_"), np.log(qcosmo30).add_prefix("lnq_")], axis=1)

means0006, cov0006 = cosmo_prior(xraso, xcosmo00, xcosmo06)
means2430, cov2430 = cosmo_prior(xraso, xcosmo24, xcosmo30)

means0006.to_csv("../data/unified/priors/x_cosmo7+00+06_mean.csv")
means2430.to_csv("../data/unified/priors/x_cosmo7+24+30_mean.csv")
cov0006.to_csv("../data/unified/priors/x_cosmo7+00+06_cov.csv")
cov2430.to_csv("../data/unified/priors/x_cosmo7+24+30_cov.csv")

## Radiosonde Climatology

First, separate test and training data. Climatology prior is then only computed from the training data.

In [ ]:
def intestdata(d):
    return dt.datetime(2015, 2, 1) < d < dt.datetime(2016, 1, 31)

### Temperature

In [ ]:
Traso = read_csv_profiles("../data/unified/T_raso.csv")
Tclim = Traso.ix[[v for v in Traso.index if not intestdata(v)]]
Ttest = Traso.ix[[v for v in Traso.index if intestdata(v)]]
Tclim.to_csv("../data/unified/training/T_rasoclim.csv")
Ttest.to_csv("../data/unified/test/T_rasoclim.csv")

In [ ]:
pd.DataFrame(Tclim.mean(), columns=["T"]).to_csv("../data/unified/priors/T_rasoclim_mean.csv")
Tclim.cov().to_csv("../data/unified/priors/T_rasoclim_cov.csv")

### Humidity

In [ ]:
qraso = read_csv_profiles("../data/unified/qvap_raso.csv")
qraso += read_csv_profiles("../data/unified/qliq_raso.csv")
qraso = np.log(qraso)
qclim = qraso.ix[[v for v in qraso.index if not intestdata(v)]]
qtest = qraso.ix[[v for v in qraso.index if intestdata(v)]]
qclim.to_csv("../data/unified/training/lnq_rasoclim.csv")
qtest.to_csv("../data/unified/test/lnq_rasoclim.csv")

In [ ]:
pd.DataFrame(qclim.mean(), columns=["lnq"]).to_csv("../data/unified/priors/lnq_rasoclim_mean.csv")
qclim.cov().to_csv("../data/unified/priors/lnq_rasoclim_cov.csv")

### State Vector

In [ ]:
xclim = pd.concat([Traso.add_prefix("T_"), qraso.add_prefix("lnq_")], axis=1)
pd.DataFrame(xclim.mean(), columns=["x"]).to_csv("../data/unified/priors/x_rasoclim_mean.csv")
xclim.cov().to_csv("../data/unified/priors/x_rasoclim_cov.csv")

### Cloudy cases

Mark cloudy profiles.

In [ ]:
raso = read_csv_profiles("../data/unified/qliq_raso.csv")
clim = raso.ix[[v for v in raso.index if not intestdata(v)]]
test = raso.ix[[v for v in raso.index if intestdata(v)]]
(clim.sum(axis=1) > 0).rename("cloudy").to_frame().to_csv("../data/unified/training/cloudy_raso.csv")
(test.sum(axis=1) > 0).rename("cloudy").to_frame().to_csv("../data/unified/test/cloudy_raso.csv")

In [ ]:
igmk = read_csv_profiles("../data/unified/cloudy_igmk.csv")
igmk = igmk.reindex(clim.index, method="nearest", tolerance=dt.timedelta(minutes=60)).dropna()
igmk = igmk.drop(dt.datetime(1999, 11, 18, 22, 51)) # foreshadowing...
igmk = igmk.drop(dt.datetime(2012, 2, 14, 3, 37)) # foreshadowing...
igmk.ix[[v for v in raso.index if not intestdata(v)]].dropna().to_csv("../data/unified/training/cloudy_igmk.csv")
# no test set b/c no data in period available

## Radiative Transfer Model

Separate training and test datasets. Then use only clear-sky cases for determination of covariance.

In [ ]:
mwrtm = read_csv_profiles("../data/unified/TB_mwrtm_6000_full_hr.csv").drop(["T", "qvap"], axis=1)
mwrtm_train = mwrtm.drop("p", axis=1).ix[[v for v in mwrtm.index if not intestdata(v)]]
mwrtm_test = mwrtm.drop("p", axis=1).ix[[v for v in mwrtm.index if intestdata(v)]]
mwrtm_train.to_csv("../data/unified/training/TB_mwrtm.csv")
mwrtm_test.to_csv("../data/unified/test/TB_mwrtm.csv")

psfc = mwrtm[["p"]]
psfc.ix[[v for v in psfc.index if not intestdata(v)]].to_csv("../data/unified/training/psfc.csv")
psfc.ix[[v for v in psfc.index if intestdata(v)]].to_csv("../data/unified/test/psfc.csv")

In [ ]:
def load_tb_data(filename):
    df = read_csv_profiles(filename).drop(["p", "T", "qvap"], axis=1)
    train = df.ix[[v for v in df.index if not intestdata(v)]]
    test = df.ix[[v for v in df.index if intestdata(v)]]
    return train, test

In [ ]:
mwrtmfap_train, mwrtmfap_test = load_tb_data("../data/unified/TB_mwrtm_2500_fap.csv")
mwrtmfap_train.to_csv("../data/unified/training/TB_mwrtm_fap.csv")
mwrtmfap_test.to_csv("../data/unified/test/TB_mwrtm_fap.csv")

monortm_train, monortm_test = load_tb_data("../data/unified/TB_monortm_hr.csv")
monortm_train.to_csv("../data/unified/training/TB_monortm.csv")
monortm_test.to_csv("../data/unified/test/TB_monortm.csv")

igmk_train, igmk_test = load_tb_data("../data/unified/TB_igmk.csv")
igmk_train = igmk_train.reindex(mwrtm_train.index, method="nearest", tolerance=dt.timedelta(minutes=60)).dropna()
igmk_train = igmk_train.drop(dt.datetime(2012, 2, 14, 3, 37)) # foreshadowing...
igmk_train.to_csv("../data/unified/training/TB_igmk.csv")
# no test set b/c no data in period available

In [ ]:
cloudy_raso = read_csv_profiles("../data/unified/training/cloudy_raso.csv")["cloudy"]
cloudy_igmk = read_csv_profiles("../data/unified/training/cloudy_igmk.csv")["cloudy"]

Determine all kinds of covariance matrices for clear-skies

In [ ]:
cov = (mwrtmfap_train.loc[~cloudy_raso,monortm_train.columns] - monortm_train.loc[~cloudy_raso,:]).dropna(axis=0).cov()
cov.to_csv("../data/unified/priors/TB_mwrtm_fap_monortm_cov.csv")

cov = (mwrtmfap_train.loc[~cloudy_raso,:] - igmk_train.ix[cloudy_igmk.index][~cloudy_igmk]).dropna(axis=0).cov()
cov.to_csv("../data/unified/priors/TB_mwrtm_fap_igmk_cov.csv")

cov = (igmk_train.ix[cloudy_igmk.index][~cloudy_igmk][monortm_train.columns] - monortm_train[~cloudy_raso]).dropna(axis=0).cov()
cov.to_csv("../data/unified/priors/TB_igmk_monortm_cov.csv")

## MWRTM - HATPRO bias

In [ ]:
tbs = read_csv_profiles("../data/unified/test/TB_hatpro.csv")
tb_mwrtm = read_csv_profiles("../data/unified/test/TB_mwrtm.csv")
tb_mwrtmfap = read_csv_profiles("../data/unified/test/TB_mwrtm_fap.csv")
tb_monortm = read_csv_profiles("../data/unified/test/TB_monortm.csv")
cloudy = read_csv_profiles("../data/unified/test/cloudy_raso.csv")

In [ ]:
tb_hatpro = tbs.reindex(tb_mwrtm.index, method="nearest", tolerance=dt.timedelta(minutes=30)).dropna()
tb_hatpro = pd.concat([tb_hatpro, cloudy], axis=1).dropna()

Due to the large uncertainties with cloud representation in the radiosonde ascents, only clear sky observations are considered for the bias calculation.

The observation from 2015-10-29 02:15:06 is classified as clear from the radiosonde but the webcam picture clearly
shows the existence of a liquid cloud layer:

<img src="http://www.foto-webcam.eu/webcam/innsbruck/2015/10/29/0420_la.jpg" width=50%>

It is therefore removed from the dataset.

In [ ]:
tb_hatpro = tb_hatpro.where(~tb_hatpro["cloudy"]).dropna().iloc[:-1,:]
len(tb_hatpro)

This is poor statistics but what else to do?

In [ ]:
(tb_mwrtm - tb_hatpro).mean().dropna().rename("bias").to_frame().to_csv("../data/unified/priors/TB_mwrtm_bias.csv")
(tb_mwrtmfap - tb_hatpro).mean().dropna().rename("bias").to_frame().to_csv("../data/unified/priors/TB_mwrtm_fap_bias.csv")
(tb_monortm - tb_hatpro).mean().dropna().rename("bias").to_frame().to_csv("../data/unified/priors/TB_monortm_bias.csv")

Bias must be subtracted.